# Changelog

Version 4
- Trying 1 week Forecast
- Implementing Streamlit

# Importing

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [4]:
user1_df = pd.read_csv("user1_presence_data_60.csv")
user2_df = pd.read_csv("user2_presence_data_60.csv")

In [5]:
# Convert 'time' column to datetime
user1_df["time"] = pd.to_datetime(user1_df["time"])
user2_df["time"] = pd.to_datetime(user2_df["time"])

In [6]:
# Merge datasets directly by time to ensure each row contains both users' states
merged_df = pd.merge(user1_df, user2_df, on="time", suffixes=("_user1", "_user2"))

# Feature Engineering

In [8]:
# Extract hour, minute, day
merged_df["hour"] = merged_df["time"].dt.hour
merged_df["minute"] = merged_df["time"].dt.minute
merged_df["day_of_week"] = merged_df["time"].dt.dayofweek  # Monday=0 ... Sunday=6
merged_df["is_weekend"] = merged_df["day_of_week"].isin([5,6]).astype(int)

In [9]:
merged_df

,time,state_user1,state_user2,hour,minute,day_of_week,is_weekend
0,2022-04-24 00:01:00+00:00,living room,living room,0,1,6,1
1,2022-04-24 00:01:30+00:00,living room,living room,0,1,6,1
2,2022-04-24 00:02:00+00:00,living room,living room,0,2,6,1
3,2022-04-24 00:02:30+00:00,living room,living room,0,2,6,1
4,2022-04-24 00:03:00+00:00,living room,living room,0,3,6,1
...,...,...,...,...,...,...,...
172793,2022-06-22 23:57:30+00:00,bedroom,bedroom,23,57,2,0
172794,2022-06-22 23:58:00+00:00,bedroom,bedroom,23,58,2,0
172795,2022-06-22 23:58:30+00:00,bedroom,bedroom,23,58,2,0
172796,2022-06-22 23:59:00+00:00,bedroom,bedroom,23,59,2,0


## Model Preprocessing

In [11]:
# Label-encode states
le_state_user1 = LabelEncoder()
le_state_user2 = LabelEncoder()

merged_df["state_user1"] = le_state_user1.fit_transform(merged_df["state_user1"])
merged_df["state_user2"] = le_state_user2.fit_transform(merged_df["state_user2"])

# Save encoders for later
with open("le_state_user1.pkl","wb") as f:
    pickle.dump(le_state_user1, f)
with open("le_state_user2.pkl","wb") as f:
    pickle.dump(le_state_user2, f)

In [12]:
# Cyclical features
merged_df["hour_sin"] = np.sin(2*np.pi*merged_df["hour"]/24)
merged_df["hour_cos"] = np.cos(2*np.pi*merged_df["hour"]/24)

merged_df["minute_sin"] = np.sin(2*np.pi*merged_df["minute"]/60)
merged_df["minute_cos"] = np.cos(2*np.pi*merged_df["minute"]/60)

merged_df["day_sin"] = np.sin(2*np.pi*merged_df["day_of_week"]/7)
merged_df["day_cos"] = np.cos(2*np.pi*merged_df["day_of_week"]/7)

In [13]:
# Drop columns not needed
merged_df.drop(columns=["time","hour","minute","day_of_week"], inplace=True)

In [14]:
merged_df

,state_user1,state_user2,is_weekend,hour_sin,hour_cos,minute_sin,minute_cos,day_sin,day_cos
0,3,3,1,0.000000,1.000000,0.104528,0.994522,-0.781831,0.623490
1,3,3,1,0.000000,1.000000,0.104528,0.994522,-0.781831,0.623490
2,3,3,1,0.000000,1.000000,0.207912,0.978148,-0.781831,0.623490
3,3,3,1,0.000000,1.000000,0.207912,0.978148,-0.781831,0.623490
4,3,3,1,0.000000,1.000000,0.309017,0.951057,-0.781831,0.623490
...,...,...,...,...,...,...,...,...,...
172793,0,0,0,-0.258819,0.965926,-0.309017,0.951057,0.974928,-0.222521
172794,0,0,0,-0.258819,0.965926,-0.207912,0.978148,0.974928,-0.222521
172795,0,0,0,-0.258819,0.965926,-0.207912,0.978148,0.974928,-0.222521
172796,0,0,0,-0.258819,0.965926,-0.104528,0.994522,0.974928,-0.222521


# Reshaping

In [16]:
num_states_user1 = len(le_state_user1.classes_)
num_states_user2 = len(le_state_user2.classes_)

In [17]:
# Suppose 24hr -> 30s intervals => 24*60*2 = 2880 time steps
time_steps = 2880

# Separate features vs states
features = merged_df.drop(columns=["state_user1","state_user2"]).values
user1_states = merged_df["state_user1"].values
user2_states = merged_df["state_user2"].values

# Keep only multiples of time_steps
num_samples = (len(features)//time_steps)*time_steps
features = features[:num_samples]
user1_states = user1_states[:num_samples]
user2_states = user2_states[:num_samples]

batch_size = num_samples//time_steps

In [18]:
# Reshape to (batch_size, time_steps, num_features)
X = features.reshape(batch_size, time_steps, features.shape[1])
y1 = user1_states.reshape(batch_size, time_steps, 1)
y2 = user2_states.reshape(batch_size, time_steps, 1)

In [19]:
print("X.shape =", X.shape, "y1.shape =", y1.shape, "y2.shape =", y2.shape)

X.shape = (59, 2880, 7) y1.shape = (59, 2880, 1) y2.shape = (59, 2880, 1)


# Building the LSTM Model

In [26]:
def create_lstm_model(input_shape, num_states_user1, num_states_user2):
    inputs = Input(shape=input_shape)

    x = LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001))(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = LSTM(32, return_sequences=True, kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Output for user1
    out_user1 = Dense(num_states_user1, activation="softmax", name="state_user1")(x)
    # Output for user2
    out_user2 = Dense(num_states_user2, activation="softmax", name="state_user2")(x)

    model = Model(inputs=inputs, outputs=[out_user1, out_user2])

    # Provide a separate metric for each output
    model.compile(
        loss=["sparse_categorical_crossentropy","sparse_categorical_crossentropy"],
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics=[["accuracy"], ["accuracy"]]
    )
    return model

In [28]:
num_states_user1 = len(le_state_user1.classes_)
num_states_user2 = len(le_state_user2.classes_)

# Model Training

In [30]:
tscv = TimeSeriesSplit(n_splits=5)

user1_reports = []
user2_reports = []

fold = 1
for train_idx, val_idx in tscv.split(X):
    print(f"\n===== FOLD {fold} / 5 =====\n")

    X_train, X_val = X[train_idx], X[val_idx]
    y1_train, y1_val = y1[train_idx], y1[val_idx]
    y2_train, y2_val = y2[train_idx], y2[val_idx]

    # Build fresh model
    model = create_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_states_user1=num_states_user1,
        num_states_user2=num_states_user2
    )

    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", 
        patience=3, 
        restore_best_weights=True
    )

    # Train
    model.fit(
        X_train, 
        {"state_user1": y1_train, "state_user2": y2_train},
        validation_data=(X_val, {"state_user1": y1_val, "state_user2": y2_val}),
        epochs=100,
        batch_size=4,
        callbacks=[early_stop]
    )

    # Evaluate on val
    y1_pred_probs, y2_pred_probs = model.predict(X_val)
    y1_pred = y1_pred_probs.argmax(axis=-1).flatten()
    y2_pred = y2_pred_probs.argmax(axis=-1).flatten()

    y1_val_flat = y1_val.flatten()
    y2_val_flat = y2_val.flatten()

    # Classification reports
    report_user1 = classification_report(y1_val_flat, y1_pred, output_dict=True)
    report_user2 = classification_report(y2_val_flat, y2_pred, output_dict=True)

    user1_reports.append(report_user1)
    user2_reports.append(report_user2)

    print("User1 Classification Report (Fold", fold, "):")
    print(classification_report(y1_val_flat, y1_pred))
    print("\nUser2 Classification Report (Fold", fold, "):")
    print(classification_report(y2_val_flat, y2_pred))

    fold += 1


===== FOLD 1 / 5 =====

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 6.0435 - state_user1_accuracy: 0.0959 - state_user1_loss: 2.8302 - state_user2_accuracy: 0.1073 - state_user2_loss: 3.0935 - val_loss: 3.9352 - val_state_user1_accuracy: 0.2173 - val_state_user1_loss: 1.9106 - val_state_user2_accuracy: 0.2155 - val_state_user2_loss: 1.9445
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 4.9849 - state_user1_accuracy: 0.1758 - state_user1_loss: 2.3189 - state_user2_accuracy: 0.1702 - state_user2_loss: 2.5456 - val_loss: 3.8692 - val_state_user1_accuracy: 0.3149 - val_state_user1_loss: 1.8839 - val_state_user2_accuracy: 0.3218 - val_state_user2_loss: 1.8977
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 4.5173 - state_user1_accuracy: 0.1782 - state_user1_loss: 2.2278 - state_user2_accuracy: 0.2553 - state_user2_loss: 2.1709 - val_loss: 3.8045 - val_state_user1_accuracy: 0.3552 - val_state_user1_loss: 1.8606 - val_state_user2_accuracy: 0.4483 - val_state_user

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packag

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 5.6898 - state_user1_accuracy: 0.1442 - state_user1_loss: 2.9666 - state_user2_accuracy: 0.1525 - state_user2_loss: 2.6152 - val_loss: 3.9248 - val_state_user1_accuracy: 0.2311 - val_state_user1_loss: 1.9471 - val_state_user2_accuracy: 0.3223 - val_state_user2_loss: 1.8972
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 4.4584 - state_user1_accuracy: 0.2381 - state_user1_loss: 2.2293 - state_user2_accuracy: 0.2809 - state_user2_loss: 2.1274 - val_loss: 3.8149 - val_state_user1_accuracy: 0.4579 - val_state_user1_loss: 1.8727 - val_state_user2_accuracy: 0.4303 - val_state_user2_loss: 1.8471
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 3.9771 - state_user1_accuracy: 0.3361 - state_user1_loss: 1.9798 - state_user2_accuracy: 0.3805 - state_user2_loss: 1.9004 - val_loss: 3.7122 - val_state_user1_accuracy: 0.5436 - val_state_user1_loss: 1.8026 - val_state_user2_accuracy: 0.4708 - val_state_user2_loss: 1.8016
Epoch 4/1

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packag

8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 940ms/step - loss: 5.2567 - state_user1_accuracy: 0.1792 - state_user1_loss: 2.5614 - state_user2_accuracy: 0.1741 - state_user2_loss: 2.5957 - val_loss: 3.8520 - val_state_user1_accuracy: 0.3547 - val_state_user1_loss: 1.8641 - val_state_user2_accuracy: 0.3709 - val_state_user2_loss: 1.8811
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 887ms/step - loss: 4.1578 - state_user1_accuracy: 0.2966 - state_user1_loss: 1.9757 - state_user2_accuracy: 0.2980 - state_user2_loss: 2.0828 - val_loss: 3.7204 - val_state_user1_accuracy: 0.5140 - val_state_user1_loss: 1.7853 - val_state_user2_accuracy: 0.5022 - val_state_user2_loss: 1.8429
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 873ms/step - loss: 3.8347 - state_user1_accuracy: 0.4029 - state_user1_loss: 1.7324 - state_user2_accuracy: 0.3627 - state_user2_loss: 2.0034 - val_loss: 3.6143 - val_state_user1_accuracy: 0.5754 - val_state_user1_loss: 1.7278 - val_state_user2_accuracy: 0.5357 - val_state_user2_loss: 1.8077
Epoch 4/100


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packag

11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: 5.0522 - state_user1_accuracy: 0.1957 - state_user1_loss: 2.3259 - state_user2_accuracy: 0.2199 - state_user2_loss: 2.6207 - val_loss: 3.7917 - val_state_user1_accuracy: 0.4488 - val_state_user1_loss: 1.8552 - val_state_user2_accuracy: 0.5501 - val_state_user2_loss: 1.8380
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 946ms/step - loss: 3.8372 - state_user1_accuracy: 0.3784 - state_user1_loss: 1.8483 - state_user2_accuracy: 0.4365 - state_user2_loss: 1.8808 - val_loss: 3.6377 - val_state_user1_accuracy: 0.5691 - val_state_user1_loss: 1.7786 - val_state_user2_accuracy: 0.6114 - val_state_user2_loss: 1.7543
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 955ms/step - loss: 3.5856 - state_user1_accuracy: 0.4843 - state_user1_loss: 1.6544 - state_user2_accuracy: 0.4801 - state_user2_loss: 1.8497 - val_loss: 3.5284 - val_state_user1_accuracy: 0.6175 - val_state_user1_loss: 1.7281 - val_state_user2_accuracy: 0.6196 - val_state_user2_loss: 1.7040
Epoch 4

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packag

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 5.3127 - state_user1_accuracy: 0.1830 - state_user1_loss: 2.4949 - state_user2_accuracy: 0.1849 - state_user2_loss: 2.7148 - val_loss: 3.7885 - val_state_user1_accuracy: 0.4260 - val_state_user1_loss: 1.8367 - val_state_user2_accuracy: 0.5414 - val_state_user2_loss: 1.8508
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: 4.0216 - state_user1_accuracy: 0.3341 - state_user1_loss: 1.9746 - state_user2_accuracy: 0.3587 - state_user2_loss: 1.9558 - val_loss: 3.6123 - val_state_user1_accuracy: 0.5770 - val_state_user1_loss: 1.7572 - val_state_user2_accuracy: 0.6716 - val_state_user2_loss: 1.7600
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - loss: 3.5305 - state_user1_accuracy: 0.4566 - state_user1_loss: 1.6745 - state_user2_accuracy: 0.4597 - state_user2_loss: 1.7594 - val_loss: 3.4682 - val_state_user1_accuracy: 0.6137 - val_state_user1_loss: 1.6816 - val_state_user2_accuracy: 0.6929 - val_state_user2_loss: 1.6949
E

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packag

In [31]:
# Compute and print average F1-scores across folds (only for class labels, not averages)
def compute_avg_f1(reports):
    avg_f1 = {}
    first_report = reports[0]

    for label in first_report.keys():
        if isinstance(first_report[label], dict) and "f1-score" in first_report[label]:  # Exclude non-dictionary values
            f1_scores = [rep[label]['f1-score'] for rep in reports if label in rep]  # Check if label exists in each report
            if f1_scores:  # Ensure the list is not empty before averaging
                avg_f1[label] = np.mean(f1_scores)
    
    return avg_f1

# Get average F1-score for User1 and User2
avg_report_user1 = compute_avg_f1(user1_reports)
avg_report_user2 = compute_avg_f1(user2_reports)

print("\n🔹 Average F1-Scores Across Folds for User1:")
for label, f1 in avg_report_user1.items():
    print(f"Class {label}: {f1:.4f}")

print("\n🔹 Average F1-Scores Across Folds for User2:")
for label, f1 in avg_report_user2.items():
    print(f"Class {label}: {f1:.4f}")


🔹 Average F1-Scores Across Folds for User1:
Class 0: 0.7072
Class 1: 0.0000
Class 2: 0.0323
Class 3: 0.0963
Class 4: 0.0000
Class 5: 0.7664
Class 6: 0.1300
Class macro avg: 0.2474
Class weighted avg: 0.6196

🔹 Average F1-Scores Across Folds for User2:
Class 0: 0.6956
Class 1: 0.0000
Class 2: 0.0000
Class 3: 0.0000
Class 4: 0.0000
Class 5: 0.6592
Class 6: 0.0000
Class macro avg: 0.1935
Class weighted avg: 0.5894


In [32]:
# Build a final model on the entire dataset
final_model = create_lstm_model(
    input_shape=(X.shape[1], X.shape[2]),
    num_states_user1=num_states_user1,
    num_states_user2=num_states_user2
)

early_stop_final = tf.keras.callbacks.EarlyStopping(
    monitor="loss",
    patience=3,
    restore_best_weights=True
)

final_model.fit(
    X,
    {"state_user1": y1, "state_user2": y2},
    epochs=100,
    batch_size=4,
    callbacks=[early_stop_final]
)

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 18s 927ms/step - loss: 5.1029 - state_user1_accuracy: 0.1496 - state_user1_loss: 2.7177 - state_user2_accuracy: 0.2736 - state_user2_loss: 2.2858
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 937ms/step - loss: 3.8539 - state_user1_accuracy: 0.3735 - state_user1_loss: 1.8755 - state_user2_accuracy: 0.4061 - state_user2_loss: 1.8815
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 934ms/step - loss: 3.3445 - state_user1_accuracy: 0.4867 - state_user1_loss: 1.5599 - state_user2_accuracy: 0.4986 - state_user2_loss: 1.6879
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 930ms/step - loss: 3.0719 - state_user1_accuracy: 0.5633 - state_user1_loss: 1.3770 - state_user2_accuracy: 0.5457 - state_user2_loss: 1.5977
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 933ms/step - loss: 2.9734 - state_user1_accuracy: 0.5679 - state_user1_loss: 1.3509 - state_user2_accuracy: 0.5612 - state_user2_loss: 1.5278
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 935ms/step - loss: 2.9371 - stat

Saved final model to full_week_lstm_model.h5


In [37]:
# Save the final single model
final_model.save("full_week_lstm_model.keras")